# Chart generations Notebook

## Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot

ModuleNotFoundError: No module named 'plotly'

In [2]:
offline.init_notebook_mode(connected=True)

## Bubble Chart for Emissions Intensity


In [3]:
def get_company_id(corp_name):
    XLSX_PATH = os.path.join ('../django_project/static/django_project', 'data', 'sp100_data.xlsx')
    df= pd.read_excel(
        XLSX_PATH, 
        sheet_name = "companies",
        engine = 'openpyxl', 
        usecols = ['company_id', 'company_name'],
        )
    return df.loc[df['company_name']==corp_name].iloc[0,0]

In [4]:
corp_name = "3M"
company_id = get_company_id(corp_name)

# Step 1: Set up the data sources

In [5]:
XLSX_PATH = os.path.join ('../django_project/static/django_project', 'data', 'sp100_data.xlsx')
COLS_TO_USE = {
    'GHG17':['company_id', 'gross_total_scope1', 'gross_scope2_calc', 'gross_total_scope3'],
    'GHG18':['company_id', 'gross_total_scope1', 'gross_scope2_calc', 'gross_total_scope3'],
    'GHG19':['company_id', 'gross_total_scope1', 'gross_scope2_calc', 'gross_total_scope3'],
}


# Step 2: Merge the tables

In [6]:
years = ["2017", "2018", "2019"]


In [7]:
i=0
all_df = []
for sheetname, cols in COLS_TO_USE.items():
    
    df_by_year = pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        )
    df_by_year = df_by_year.loc[df_by_year['company_id'] == company_id]
    df_by_year['year'] = years[i]

    all_df.append(df_by_year)
    i+=1

In [8]:
    merged_df = pd.concat(
        all_df
    )

In [9]:
merged_df.head()

,company_id,gross_total_scope1,gross_total_scope3,gross_scope2_calc,year
0,1.0,3960000,9280000,1650000.0,2017
0,1.0,4790000,9530000,1480000.0,2018
0,1.0,4050000,8.9258e+06,1320000.0,2019


In [10]:
bar_data = merged_df[['gross_total_scope1','gross_scope2_calc', 'gross_total_scope3']]

# Extract each data serie

In [11]:
year = merged_df['year']
scope1 = pd.to_numeric(merged_df['gross_total_scope1'])
scope2 = pd.to_numeric(merged_df['gross_scope2_calc'])
scope3 = pd.to_numeric(merged_df['gross_total_scope3'])
bar_data = pd.concat([year, scope1, scope2, scope3], axis=1, ignore_index=False)


In [12]:
bar_data

,year,gross_total_scope1,gross_scope2_calc,gross_total_scope3
0,2017,3960000,1650000.0,9280000
0,2018,4790000,1480000.0,9530000
0,2019,4050000,1320000.0,8925800


In [14]:
trace1 = go.Bar(
    x = years,
    y = scope1,
    name = 'Scope1',
    marker_color = '#488f31'
    )
trace2 = go.Bar(
    x = years,
    y = scope2,
    name = 'Scope2',
    marker_color='#feba65',
    )
trace3 = go.Bar(
    x = years,
    y = scope3,
    name = 'Scope3',
    marker_color='#de425b',
    )

data = [trace1, trace2, trace3]

layout = go.Layout (
    barmode='stack',
    title = '<b>GHG Emissions Evolution</b><br>',
    title_x = 0.5,
    titlefont = dict(
        family = 'Arial',
        size = 16),
        paper_bgcolor="#bad0af",
        plot_bgcolor = 'antiquewhite',
        xaxis =  dict(
        title = '<b>Reporting years</b>'),
        yaxis = dict(
            title = 'Emissions (in tons of CO2e)'),    
    legend=dict(
        title="Emissions",
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="left",
        x=0),
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=80,
        pad=4
    )
    
)

config = {'displaylogo': False}


fig = go.Figure(
    data=data,
    layout = layout
    )


In [15]:
offline.iplot(fig, config = config)

In [16]:
name_fig = "ghg_barchart_{}".format(corp_name)
#fig.write_image("../django_project/static/django_project/images/charts/{}.svg".format(name_fig), scale=3, height = 300)
plot(fig, config=config, filename = '../django_project/templates/django_project/corporates/charts/html_exports/{}.html'.format(name_fig), auto_open=False)

'../django_project/templates/django_project/corporates/charts/html_exports/ghg_barchart_3M.html'